In [33]:
import pandas as pd

In [34]:
df =pd.read_csv('/content/anime.csv')

In [35]:
df.shape

(12294, 7)

In [36]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [37]:
# Convert object data type to string
df['name'] = df['name'].astype(str)
df['type'] = df['type'].astype(str)
df['genre'] = df['genre'].astype(str)


In [38]:
#handling missing values
df['rating'].fillna(df['rating'].mean(),inplace=True)


# handling missing categorical values
#note: you can also drop the rows  depending on  your data
df['genre'].fillna(df['genre'].mode()[0], inplace=True)
df['type'].fillna(df['type'].mode()[0],inplace=True)


In [39]:
df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [40]:
df.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12294 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [41]:
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [42]:
# Convert 'genre' to list of genres
df['genre'] = df['genre'].str.split(', ')


In [43]:
# One-hot encode the 'genre' column
anime_genres = df['genre'].explode().unique()
for genre in anime_genres:
    df[genre] = df['genre'].apply(lambda x: 1 if genre in x else 0)

# Drop the original 'genre' column
df.drop('genre', axis=1, inplace=True)

In [44]:
# One-hot encode the 'type' column
df = pd.get_dummies(df, columns=['type'])


In [46]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the rating and members columns
scaler = MinMaxScaler()
df[['rating', 'members']] = scaler.fit_transform(df[['rating', 'members']])


In [47]:
from sklearn.metrics.pairwise import cosine_similarity

# Use relevant features for similarity calculation
features = df.drop(['anime_id', 'name', 'episodes'], axis=1)

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(features, features)


In [51]:
def get_recommendations(anime_title, df, cosine_sim, top_n=10):
    # Find the index of the anime
    idx = df.index[df['name'] == anime_title].tolist()[0]

    # Get the pairwise similarity scores of all animes with that anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the animes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top-n most similar animes
    sim_scores = sim_scores[1:top_n+1]
    anime_indices = [i[0] for i in sim_scores]

    # Return the top-n most similar animes
    return df['name'].iloc[anime_indices]

# Example usage
print(get_recommendations('Hunter x Hunter (2011)', df, cosine_sim))


112            Hunter x Hunter
9673             Nano Invaders
1796            Rekka no Honoo
3757          Eat-Man &#039;98
4521                   Eat-Man
4548            Beast Wars Neo
1887         Allison to Lillia
5028    Kouya no Shounen Isamu
7125            Big Order (TV)
8640                Gene Diver
Name: name, dtype: object


**Can you explain the difference between user-based and item-based collaborative filtering?**

Item based filtering finds the similiary between the items you consume and user based filtering finds the user with similar consumption pattern as  yourself and gives you content that  users found interesting

**What is collaborative filtering, and how does it work?**

Collaborative filtering is family of algorithms  where there are multiple ways to find similar users or items.
In collaborative filtering the similarity is based on rating the user gives. If both users give same rating they are considered similar.